In [1]:
import torch
import numpy as np
import math
import pickle 
from tqdm import tqdm



In [16]:
import h5py#prepare data original
import os
import pickle
from tqdm import tqdm

from scipy import io
import numpy as np
import torch
from sklearn.model_selection import train_test_split

from torch.utils.data import DataLoader

import json


NUM_FEAT = np.int32(10000)

# Define constants
BATCH_SIZE = 128
TRAIN_RATIO = 0.8

# Load MATLAB data
matlab_data = io.loadmat('NumStim_1to20_100x100_TR.mat')

D = matlab_data['D']  # Images data
N_list = matlab_data['N_list'].flatten()  # Numerosity list

# Shuffle the data
D_train, D_test, N_list_train, N_list_test = train_test_split(D.T, N_list, test_size=0.2, random_state=42)

train_data = list()
train_labels = list()
# Process the train data
num_batches_train = len(N_list_train) // BATCH_SIZE
train_data = torch.empty(num_batches_train, BATCH_SIZE, 10000)
train_labels = torch.empty(num_batches_train, BATCH_SIZE, 1)

data = D_train
labels = N_list_train

catdata = np.hstack((data, labels.reshape((-1, 1))))
np.random.shuffle(catdata)

data = catdata[:, :NUM_FEAT]
labels = catdata[:, -1].flatten()

# Iterate over train batches
with tqdm(range(num_batches_train), unit='Batch') as tdata:
    for n in tdata:
        tdata.set_description(f'Train Batch {n}\t')
        train_data[n, :, :] = torch.Tensor(data[n: n + BATCH_SIZE, :]).type(torch.float32)
        unsqz_labels = torch.Tensor(labels[n: n + BATCH_SIZE]).type(torch.float32)
        train_labels[n, :, :] = unsqz_labels.reshape(-1, 1)

data = D_test
labels = N_list_test

catdata = np.hstack((data, labels.reshape((-1, 1))))
np.random.shuffle(catdata)

data = catdata[:, :NUM_FEAT]
labels = catdata[:, -1].flatten()

test_data = list()
test_labels = list()

# Process the test data
num_batches_test = len(N_list_test) // BATCH_SIZE
test_data = torch.empty(num_batches_test, BATCH_SIZE, NUM_FEAT)
test_labels = torch.empty(num_batches_test, BATCH_SIZE, 1)

# Iterate over test batches
with tqdm(range(num_batches_test), unit='Batch') as tdata:
    for n in tdata:
        tdata.set_description(f'Test Batch {n}\t')
        test_data[n, :, :] = torch.Tensor(data[n: n + BATCH_SIZE, :]).type(torch.float32)
        unsqz_labels = torch.Tensor(labels[n: n + BATCH_SIZE]).type(torch.float32)
        test_labels[n, :, :] = unsqz_labels.reshape(-1, 1)
        # end
# Define the path to save the processed data
path_dump_data = (r'C:\Users\fraca\Desktop\Groundeep\latestcode')

if not os.path.exists(path_dump_data):
    os.mkdir(path_dump_data)

# Save the processed data
train_dataset = {'data': train_data, 'labels': train_labels}
test_dataset = {'data': test_data, 'labels': test_labels}

NUM_CHANNELS = 1  # Assuming grayscale images, change to 3 for RGB
IMAGE_WIDTH = 100  # Change this to the width of your images
IMAGE_HEIGHT = 100  # Change this to the height of your images

# Reshape train data
train_data_reshaped = train_data.view(-1, NUM_CHANNELS, IMAGE_WIDTH, IMAGE_HEIGHT)

# Reshape test data
test_data_reshaped = test_data.view(-1, NUM_CHANNELS, IMAGE_WIDTH, IMAGE_HEIGHT)

# Save the reshaped data
train_dataset = {'data': train_data_reshaped, 'labels': train_labels}
test_dataset = {'data': test_data_reshaped, 'labels': test_labels}

print(train_dataset['data'].shape)
print(test_dataset['data'].shape)

pickle.dump(
    train_dataset,
    open(os.path.join(path_dump_data, 'train_datasetx.pkl'), 'wb')
)

pickle.dump(
    test_dataset,
    open(os.path.join(path_dump_data, 'test_datasetx.pkl'), 'wb')
)


Test Batch 51	: 100%|██████████| 52/52 [00:00<00:00, 1334.22Batch/s]


torch.Size([27008, 1, 100, 100])
torch.Size([6656, 1, 100, 100])


In [2]:
params = {
     "LEARNING_RATE": 0.15,
     "WEIGHT_PENALTY": 0.0001,
     "INIT_MOMENTUM": 0.7,
     "FINAL_MOMENTUM": 0.97,  # Final momentum for dynamic momentum increase
     "LEARNING_RATE_DYNAMIC": True,
}

# Load preprocessed training dataset
with open(r'C:\Users\fraca\Desktop\Groundeep\latestcode\batched_train_data_from_mat.pkl', 'rb') as f:
    train_dataset = pickle.load(f)

 # Convert NumPy array to PyTorch tensor
train_data = torch.tensor(train_dataset['data'], dtype=torch.float32)
train_data = train_data.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

